In [1]:
! pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.4/230.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.11.0-py3-none-any.whl size=212399 sha256=40fbc5ddc36498378d44c75087c54ac6519f5b5bfbe3168403cee6b230a61f7d
  Stored in directory: /root/.cache/pip/wheels/19/00/29/5a29bbb5ebced37cb04e5d93102dd62e765f7a4e9ca3481036
Successfully built pyTelegramBotAPI


In [2]:
! pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# SQLAlchemy

Начнем с работы с базами данных через Python. Будем использовать библиотеку [SQLAlchemy](https://docs.sqlalchemy.org/en/20/)

Эта библиотека позволяет работать с базами данных с помощью питоновских объектов, что позволяет не писать сырые SQL-запросы.

Рассмотрим сперва императивное представление таблиц.

Создадим две таблицы в нашей базе данных --- таблицу "Авторы" и таблицу "Книги".

In [3]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey

meta = MetaData() # аккумилирует все данные базы данных

authors = Table('Authors', meta,
                Column('id', Integer, primary_key=True),
                Column('name', String, nullable=False, unique=True),
                )

books = Table('Books', meta,
              Column('id', Integer, primary_key=True),
              Column('title', String, nullable=False, unique=True),
              Column('author_id', String, ForeignKey('Authors.id')),
              Column('genre', String, nullable=False),
              Column('price', Integer)
              )

Чтобы теперь подсоединиться к базе, нужно использовать движок (объект ```Engine```) c помощью функции ```create_engine```.

На вход ей нужно дать строку определенного формата в зависимости от самой вашей базы данных. В этом примере использует SQLite.

In [8]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///database.db', echo=True) # database.db --- сама база даных, echo -- булевый флаг, который отвечает за сохранение SQL логов в стандартный вывод

conn = engine.connect() # коннектор, который выполняет все операции с базой

meta.create_all(engine) # чтобы создать все таблицы, нужно передать объекту meta сам engine

2023-04-16 12:48:52,703 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-16 12:48:52,707 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Authors")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Authors")


2023-04-16 12:48:52,711 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Books")


2023-04-16 12:48:52,717 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2023-04-16 12:48:52,720 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Теперь попробуем добавить в базу элементы. Напишем функцию, которая добавляет нового автора.

In [9]:
def add_new_author(name: str):
    insert_author_query = authors.insert().values(name=name) # сам SQL-запрос

    conn.execute(insert_author_query) # выполнение запроса

    conn.commit() # сохранение изменений

add_new_author('Mark Lutz')

2023-04-16 12:48:55,866 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-16 12:48:55,873 INFO sqlalchemy.engine.Engine INSERT INTO "Authors" (name) VALUES (?)


INFO:sqlalchemy.engine.Engine:INSERT INTO "Authors" (name) VALUES (?)


2023-04-16 12:48:55,877 INFO sqlalchemy.engine.Engine [generated in 0.01269s] ('Mark Lutz',)


INFO:sqlalchemy.engine.Engine:[generated in 0.01269s] ('Mark Lutz',)


2023-04-16 12:48:55,882 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Теперь напишем функцию, которая добавляет новую книгу.

In [11]:
def add_new_book(title: str, author_id: int, genre: str, price: int):
    insert_book_query = books.insert().values(title=title, author_id=author_id, genre=genre,
                                              price=price
                                              )

    conn.execute(insert_book_query)

    conn.commit()
  
add_new_book(title='Learning Python Tom 1', author_id='1', genre='Education', price=3000)

add_new_book(title='Learning Python Tom 2', author_id='1', genre='Education', price=4000)

2023-04-16 12:52:52,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-16 12:52:52,035 INFO sqlalchemy.engine.Engine INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)


2023-04-16 12:52:52,042 INFO sqlalchemy.engine.Engine [generated in 0.01549s] ('Learning Python Tom 1', '1', 'Education', 3000)


...

INFO:sqlalchemy.engine.Engine:INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)


2023-04-16 12:52:52,068 INFO sqlalchemy.engine.Engine [cached since 0.04162s ago] ('Learning Python Tom 2', '1', 'Education', 4000)


INFO:sqlalchemy.engine.Engine:[cached since 0.04162s ago] ('Learning Python Tom 2', '1', 'Education', 4000)


2023-04-16 12:52:52,071 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Попробуем теперь добавить книгу, которая уже есть.

In [16]:
try:
    add_new_book(title='Learning Python Tom 1', author_id='1', genre='Education', price=3000)

except Exception as ex:
    print()
    print(f'Exception:\n{ex}!!!')
    print()

2023-04-16 12:55:11,331 INFO sqlalchemy.engine.Engine INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)


2023-04-16 12:55:11,335 INFO sqlalchemy.engine.Engine [cached since 139.3s ago] ('Learning Python Tom 1', '1', 'Education', 3000)


INFO:sqlalchemy.engine.Engine:[cached since 139.3s ago] ('Learning Python Tom 1', '1', 'Education', 3000)



Exception:
(sqlite3.IntegrityError) UNIQUE constraint failed: Books.title
[SQL: INSERT INTO "Books" (title, author_id, genre, price) VALUES (?, ?, ?, ?)]
[parameters: ('Learning Python Tom 1', '1', 'Education', 3000)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)!!!



Теперь посмотрим, сохранились ли наши объекты в базе, с помощью ```select```.

In [19]:
from sqlalchemy import select

def select_query():
    books_select_query = books.select().where(books.c.price >= 4000)

    print()
    print(f'SQL-запрос:\n{books_select_query}') # посмотрим на сам запрос
    print()

    result = conn.execute(books_select_query)

    print()
    print(f'Результат запроса:\n {result}') # посмотрим на результат запроса
    print()

    for row in result:
        print(row)

    result = conn.execute(books_select_query).all() # просто select
    
    print()
    print(f'Результат запроса:\n {result}')
    print()

    select_query = select(books, authors).where(books.c.author_id == authors.c.id) # select с фильтрацией

    result = conn.execute(select_query).all()

    print()
    print(f'Результат запроса:\n {result}')
    print()

In [20]:
select_query()


SQL-запрос:
SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price 
FROM "Books" 
WHERE "Books".price >= :price_1

2023-04-16 13:04:05,399 INFO sqlalchemy.engine.Engine SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price 
FROM "Books" 
WHERE "Books".price >= ?


INFO:sqlalchemy.engine.Engine:SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price 
FROM "Books" 
WHERE "Books".price >= ?


2023-04-16 13:04:05,404 INFO sqlalchemy.engine.Engine [cached since 420.4s ago] (4000,)


INFO:sqlalchemy.engine.Engine:[cached since 420.4s ago] (4000,)



Результат запроса:

(2, 'Learning Python Tom 2', '1', 'Education', 4000)
2023-04-16 13:04:05,413 INFO sqlalchemy.engine.Engine SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price 
FROM "Books" 
WHERE "Books".price >= ?


...


Результат запроса:
 [(2, 'Learning Python Tom 2', '1', 'Education', 4000)]

2023-04-16 13:04:05,420 INFO sqlalchemy.engine.Engine SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price, "Authors".id AS id_1, "Authors".name 
FROM "Books", "Authors" 
WHERE "Books".author_id = "Authors".id


INFO:sqlalchemy.engine.Engine:SELECT "Books".id, "Books".title, "Books".author_id, "Books".genre, "Books".price, "Authors".id AS id_1, "Authors".name 
FROM "Books", "Authors" 
WHERE "Books".author_id = "Authors".id


2023-04-16 13:04:05,423 INFO sqlalchemy.engine.Engine [cached since 420.4s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 420.4s ago] ()



Результат запроса:
 [(1, 'Learning Python Tom 1', '1', 'Education', 3000, 1, 'Mark Lutz'), (2, 'Learning Python Tom 2', '1', 'Education', 4000, 1, 'Mark Lutz')]



Все сохранилось!

# pyTelegramBotAPI

Давайте теперь посмотрим на библиотеку, которая позволяет писать телеграм чат-ботов на Python.

Будем использовать библиотеку [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). Ее официальная документация на [русском](https://pytba.readthedocs.io/ru/latest/index.html) и на [английском](https://pytba.readthedocs.io/en/latest/index.html).

Рассмотрим прям из документации.

In [22]:
from telebot import TeleBot

BOT_TOKEN: str = '' # НИКОГДА не выкладывайте токен бота в публичный доступ

bot = TeleBot(BOT_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am EchoBot.
I am here to echo your kind words back to you. Just say anything nice and I'll say the exact same thing to you!\
""")


# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)

Давайте поменяем приветствие.

In [29]:
from telebot.types import Message

WELCOME: str = '''Привет! Это телеграм бот мастерской по Python!
Советую нажать для начала на /something :)
А чтобы познакомиться, можешь нажать на /reg 
Если хочешь почитать что-нибудь по Python, нажми /all_books!'''


@bot.message_handler(commands=['help', 'start'])
def send_welcome(message: Message):
    bot.send_message(chat_id=message.chat.id, text=WELCOME)

Добавим теперь регистрацию пользователя у нашего бота.

Создадим класс ```User``` и будем хранить всех юзеров в списке ```users```.

In [30]:
class User:
    def __init__(self, first_name: str, last_name: str, age: int, chat_id: int):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
        self.chat_id = chat_id

    def __repr__(self):
        return f'User(first name: {self.first_name}, last_name: {self.last_name}, age: {self.age})'


users = []
new_user = None

Добавим тег [/reg](), по которому будет начинаться регистрация пользователя.

In [31]:
@bot.message_handler(commands=['reg'])
def reg(message: Message):
    bot.send_message(chat_id=message.chat.id,
                     text='Введи свое имя, фамилию и возраст в формате фамилия имя возраст. Например, Муниев Аркадий 30 лет')

    bot.register_next_step_handler(message=message, callback=signup)

Функция ```signup``` нужна, чтобы обработать данные пользователя, которые он ввел.

Если он ввел их правильно, то бот отправит уточняющий вопрос с двумя кнопками, правильно ли бот записал эти данные.

In [35]:
from telebot import types # чтобы добавлять кнопки

@bot.message_handler(content_types=['text'])
def signup(message: Message):
    parse_message = message.text.split()

    if len(parse_message) <= 3:
        bot.send_message(chat_id=message.chat.id, text='Чего-то не хватает :(')
        bot.register_next_step_handler(message=message, callback=signup)

    elif len(parse_message) == 4:

        try:
            first_name, last_name, age = parse_message[0], parse_message[1], int(parse_message[2])

            global new_user

            new_user = User(first_name=first_name, last_name=last_name, age=age, chat_id=message.chat.id)

            keyboard = types.InlineKeyboardMarkup()  # клавиатура

            key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes')  # кнопка «Да»
            keyboard.add(key_yes)  # добавление кнопки в клавиатуру

            key_no = types.InlineKeyboardButton(text='Нет', callback_data='no')  # кнопка «Нет»
            keyboard.add(key_no)  # добавление кнопки в клавиатуру

            question = f'Тебя зовут {first_name}, {last_name} и тебе {age} лет?'

            bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

        except TypeError:

            bot.send_message(chat_id=message.chat.id, text='Неправильно ввел возраст')

            bot.register_next_step_handler(message=message, callback=signup)

    else:
        bot.send_message(chat_id=message.chat.id, text='Слишком много параметров')

        bot.register_next_step_handler(message=message, callback=signup)

Обработчик ответа на нажатие кнопки.

In [34]:
from telebot.types import CallbackQuery

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call: CallbackQuery):
    if call.data == "yes":  # call.data это callback_data, которую мы указали при объявлении кнопки
        users.append(new_user)

        print(users)

        bot.send_message(chat_id=call.message.chat.id, text='Запомню :)')

    elif call.data == "no":

        bot.send_message(chat_id=call.message.chat.id, text='Введи еще раз свое имя, фамилию и возраст')

        bot.register_next_step_handler_by_chat_id(call.message.chat.id, callback=signup)

Добавим еще тег [/info](), который будет отправлять кнопки со всеми ссылами на наши материалы.

In [25]:
@bot.message_handler(commands=['info'])
def info(message):
    key_board = types.InlineKeyboardMarkup(
            [
                [types.InlineKeyboardButton(text='GitHub',
                                            url='https://github.com/arikmaster22/workshop-python'
                                            )
                 ],

                [types.InlineKeyboardButton(text='Notion',
                                            url='https://educated-ambert-8c7.notion.site/Python-f340b5dcdad248f2acc5a26afd79e7ec'
                                            )
                 ],
            ]
        )

Добавим загадочный тег [/something](), по которому бот отправляет сообщение с ссылкой.

In [36]:
@bot.message_handler(commands=['something'])
def send_something(message: Message):
    bot.send_message(chat_id=message.chat.id,
                     text="<a href='https://www.youtube.com/watch?v=dQw4w9WgXcQ'>Тыкни!</a>", parse_mode='HTML',
                     disable_web_page_preview=True)

И наконец, чтобы свяжем с базой нашего бота через тег [/all_books]().

In [37]:
def parse_books(data):
    parsed_books: list[str | None] = [None] * len(data)
    for i in range(len(data)):
        select_query_author_name = select(authors.c.name).where(data[i][2] == authors.c.id)

        author_name = conn.execute(select_query_author_name).scalar()

        parsed_books[i] = f'Title: {data[i][1]}\nAuthor: {author_name}\nPrice: {data[i][4]}\nGenre: {data[i][3]}'

    return parsed_books 

@bot.message_handler(commands=['all_books'])
def send_books(message: Message):
    select_query = select(books)

    result = conn.execute(select_query).all()

    parsed_books = parse_books(result)

    for book in parsed_books:
        bot.send_message(chat_id=message.chat.id, text=book)